<a href="https://colab.research.google.com/github/AnkitaSK/Proj3_RoboReviews/blob/main/ReviewsSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import polars as pl
from transformers import pipeline

In [29]:
print(pl.__version__)

1.9.0


In [30]:
!pip install --upgrade polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 28.6 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 1.9.0
    Uninstalling polars-1.9.0:
      Successfully uninstalled polars-1.9.0


In [48]:
# Load large CSV file
df = pl.read_csv("reviews_with_Positive_topics.csv")

In [66]:
df_filtered = df.filter(pl.col("Topic") != -1)

In [67]:
print(df_filtered.head)

<bound method DataFrame.head of shape: (13_687, 27)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────────┬───────┐
│ id         ┆ dateAdded  ┆ dateUpdate ┆ name      ┆ … ┆ sourceURL ┆ reviews.r ┆ Sentiment ┆ Topic │
│ ---        ┆ ---        ┆ d          ┆ ---       ┆   ┆ s         ┆ ating_upd ┆ ---       ┆ ---   │
│ str        ┆ str        ┆ ---        ┆ str       ┆   ┆ ---       ┆ ate       ┆ str       ┆ i64   │
│            ┆            ┆ str        ┆           ┆   ┆ str       ┆ ---       ┆           ┆       │
│            ┆            ┆            ┆           ┆   ┆           ┆ i64       ┆           ┆       │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════════╪═══════╡
│ AVpgNzjwLJ ┆ 2015-10-30 ┆ 2019-04-25 ┆ AmazonBas ┆ … ┆ https://w ┆ 2         ┆ Positive  ┆ 5     │
│ eJML43Kpxn ┆ T08:59:32Z ┆ T09:08:16Z ┆ ics AAA   ┆   ┆ ww.barcod ┆           ┆           ┆       │
│            ┆            ┆            

In [68]:
print(type(df_filtered))

<class 'polars.dataframe.frame.DataFrame'>


In [26]:
print(df_filtered.columns)

['id', 'dateAdded', 'dateUpdated', 'name', 'asins', 'brand', 'categories', 'primaryCategories', 'imageURLs', 'keys', 'manufacturer', 'manufacturerNumber', 'reviews.date', 'reviews.dateSeen', 'reviews.didPurchase', 'reviews.doRecommend', 'reviews.id', 'reviews.numHelpful', 'reviews.rating', 'reviews.sourceURLs', 'reviews.text', 'reviews.title', 'reviews.username', 'sourceURLs', 'reviews.rating_update', 'Sentiment', 'Topic']


In [69]:
# count the occurence of each topic
topic_counts = df_filtered.group_by("Topic").agg(pl.col("Topic").count().alias("count"))

# Sort by the count in descending order
topic_counts = topic_counts.sort("count", descending=True)
print(topic_counts)

shape: (46, 2)
┌───────┬───────┐
│ Topic ┆ count │
│ ---   ┆ ---   │
│ i64   ┆ u32   │
╞═══════╪═══════╡
│ 0     ┆ 3774  │
│ 1     ┆ 1172  │
│ 2     ┆ 648   │
│ 3     ┆ 624   │
│ 4     ┆ 554   │
│ …     ┆ …     │
│ 41    ┆ 90    │
│ 42    ┆ 85    │
│ 43    ┆ 83    │
│ 44    ┆ 83    │
│ 45    ┆ 80    │
└───────┴───────┘


In [70]:
# Step 2: Select the top_n topics
top_n = 7
top_topics = topic_counts.head(top_n)["Topic"]

In [72]:
# Step 3: Summarize reviews for each topic
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
summaries = {}

for topic in top_topics:
    # Filter reviews for the current topic
    filtered_reviews = df_filtered.filter(df_filtered["Topic"] == topic)["reviews.text"].to_list()

    # Concatenate reviews into a single string for summarization
    concatenated_reviews = " ".join(filtered_reviews)

    # Tokenize and ensure it's within the token limit (e.g., 1024 tokens)
    inputs = summarizer.tokenizer(concatenated_reviews, return_tensors="pt", truncation=True, max_length=1024, padding="max_length")


    # Summarize the concatenated reviews (using the summarizer)
    summary = summarizer.model.generate(inputs["input_ids"], max_length=50, min_length=10, num_beams=2, early_stopping=True)

    # Decode the summary
    summary_text = summarizer.tokenizer.decode(summary[0], skip_special_tokens=True)


    summaries[topic] = summary_text


# Print summaries
print(summaries)



{0: 'AmazonBasics AA AAA batteries have done well by me appear to have a good shelf life. Will buy again.Seem to work as well as name brand batteries at a much better price These batteries are very long lasting the price is', 1: "I like the Amazon line of Basics. Nothing fancy, they just work. Can't go wrong with Amazon basics. Great bang for your buck. Greatest deal on the planet! Cheaper than my local box store. Great value.", 2: 'The Amazon Echo is a really fun thing to play with. We always try to find new things to say to it which would make it say something funny. The Echo has been the most helpful in our home for creating shopping lists.', 3: 'The modified Android operating system with Amazon features built-in is great. It has a 1.3 gigahertz processor, meaning for such a small device that decently enough. Internal memory is 8 gigs but you can expand to', 4: "They don't last as long as the brand name but are good enough considering they are much cheaper. They seem to work okay so

In [73]:
# Step 4: Print the summaries
for topic, summary in summaries.items():
   print(f"Topic {topic}: {summary}")

Topic 0: AmazonBasics AA AAA batteries have done well by me appear to have a good shelf life. Will buy again.Seem to work as well as name brand batteries at a much better price These batteries are very long lasting the price is
Topic 1: I like the Amazon line of Basics. Nothing fancy, they just work. Can't go wrong with Amazon basics. Great bang for your buck. Greatest deal on the planet! Cheaper than my local box store. Great value.
Topic 2: The Amazon Echo is a really fun thing to play with. We always try to find new things to say to it which would make it say something funny. The Echo has been the most helpful in our home for creating shopping lists.
Topic 3: The modified Android operating system with Amazon features built-in is great. It has a 1.3 gigahertz processor, meaning for such a small device that decently enough. Internal memory is 8 gigs but you can expand to
Topic 4: They don't last as long as the brand name but are good enough considering they are much cheaper. They seem